# User-based Collaborative filtering

MovieLens 데이터셋을 이용해 가장 기본적인 Collaborative Filtering을 다뤄보겠습니다.

* 유저가 평가하지 않은 영화의 점수를 user-item matrix를 이용해 예측
* 유사도는 cosine similarity와 Pearson correlation coefficient 두 가지를 이용해서 측정

In [1]:
import numpy as np
import pandas as pd
from scipy.spatial.distance import cosine

## Loading and preprocess the datasets

MovieLens 데이터는 가장 작은 100K를 사용할 것입니다.

In [2]:
path = './datasets/'
ratings = np.array(pd.read_csv(path + "ratings.csv"))#.astype(int)

u_count = int(max(ratings[:,0])+1) # number of users
m_count = int(max(ratings[:,1])+1) # number of movies


# 유저-아이템 행렬
r_mat = np.zeros([u_count,m_count])
        
for i in range(ratings.shape[0]):
    r_mat[int(ratings[i][0]),int(ratings[i][1])] = ratings[i][2]

# 유저별 rating 평균
r_mean = np.zeros([u_count])

for i in range(u_count):
    r_mean[i] = np.mean(r_mat[i,np.nonzero(r_mat[i])]) 

/Users/kwoncheol/deep/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2957: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/Users/kwoncheol/deep/lib/python3.6/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


## Normalize technique 1

유저별로 각 rating값에서 자신의 평균을 빼줍니다.

cosine similarity사용시 rating하지 않은 아이템을 negative로 처리하는 것을 방지해줍니다.

In [3]:
for i in range(u_count):
    r_mat[i,r_mat[i] != 0] = r_mat[i,r_mat[i] != 0] - r_mean[i]

## Normalize technuque 2    
      
rating이 3보다 크면 1, 3보다 작거나 평가를 하지 않았으면 0으로 값을 변경합니다.
이 예제에서는 사용하지 않습니다.

In [4]:
#for i in range(self.ratings.shape[0]):
#    self.r_mat[self.ratings[i][0],self.ratings[i][1]] = 0 if self.ratings[i][2]>=3 else 1

## Define Pearson Correlation measure function

Scipy에 PCC 함수가 있지만 직접 정의해서 사용하겠습니다. 
유사도를 측정하려는 두 유저의 인덱스를 받아 PCC값을 반환하는 함수입니다.

In [5]:
def my_pears(a, b): 
    '''
    Pearson Correlation Coefficient function
        
    Arguments:
        a - Index of user A
        b - Index of user B

    '''
    # 해당 유저가 rating한 item들을 찾기 쉽게 행렬을 뒤집어줍니다.
    
    aidx = ratings.T[0] == a
    bidx = ratings.T[0] == b

    br_idx = np.intersect1d(ratings[aidx][:,1],ratings[bidx][:,1]).astype(int)

    # 교집합이 없다면 0 반환
        
    if(len(br_idx) == 0):
        return 0

    sim = np.sum((r_mat[a,br_idx]-r_mean[a])*(r_mat[b,br_idx]-r_mean[b])) \
        / (np.sqrt(np.sum(np.square(r_mat[a,br_idx]-r_mean[a]))) \
        * np.sqrt(np.sum(np.square(r_mat[b,br_idx]-r_mean[b]))))

    return sim

## Predict users' unknown ratings

특정 유저와 특정 영화에 대한 rating 점수를 예측하는 함수를 만들겠습니다.
1. 특정 영화의 rating 점수를 갖고 있는 유저들과의 유사도를 측정해 가장 유사한 유저 K명을 구합니다
2. K명의 유저들이 그 영화에 어떤 rating을 주었는지를 이용해 우리가 원하는 유저의 rating을 예측합니다

### 점수예측은 두 가지 방법을 사용합니다.
1) 유사한 K명이 해당 영화에 준 점수를 합한 후 K로 나눕니다.

2) 각 유저와의 유사도와 그 유저들이 영화에 준 점수를 곱한 후 이를 모두 합합니다. 그 값을 유저들의 유사도의 합으로 나눕니다.

In [6]:
def predict_score(u, mov, k=10, sim_met='pearson'):
    '''
    Prediction for item 'mov' of user 'u'
        
    Arguments:
        u - 점수를 예측하고자 하는 유저  
        mov - 유저의 점수를 예측하고자 하는 영화
        k - predict에 사용할 유사한 유저의 수 (Default = 10)
            
        sim_met - 유사도측정에 사용할 함수 (Default = 'pearson')
              

    '''
    
    # Find users who have rated 'mov'
    rated_u = np.array(np.nonzero(r_mat[:,mov]))
        
    if rated_u.shape[1] == 0 or (rated_u.shape[1] == 1 and rated_u[0,0] == u):
        #print("No users who have rated this movie")
        return (0,0)
        
    rated_u = rated_u.flatten()
        
    sims = {}
    for i in range(rated_u.shape[0]):
        if rated_u[i] == u:
            continue
            
        if sim_met is 'pearson':
            sim = my_pears(u,rated_u[i])
        else:
            sim = cosine(r_mat[u,:],r_mat[rated_u[i],:])

        if len(sims) < k:
            sims[rated_u[i]] = sim

        # 현재 유저 i와의 유사도가 sims 내의 가장 유사도가 작은 유저보다 크다면 가작 작은 값을 가진 유저를 지우고
        # 현재 유저 i를 sims에 넣는다
        elif sim > sims[min_val]:
            sims.pop(min_val)
            sims[rated_u[i]] = sim

        min_val = min(sims, key=lambda k:sims[k])


    p_ver_1 = np.sum(r_mat[[*sims],mov]) / (rated_u.shape[0] if rated_u.shape[0] < k else k)
    p_ver_2 = np.sum(r_mat[[*sims],mov] * list(sims.values())) / np.sum(list(sims.values()))

    return p_ver_1, p_ver_2

## Example

유저3을 예로 직접 rating한 값과 collaborative filtering을 이용해 측정한 값을 비교해보겠습니다.

In [7]:
# 유저3이 점수를 매긴 영화들을 찾는다
rated_list = np.array(np.nonzero(r_mat[3,:])).flatten()
rated_list

array([   60,   110,   247,   267,   296,   318,   355,   356,   377,
         527,   588,   592,   593,   595,   736,   778,   866,  1197,
        1210,  1235,  1271,  1378,  1580,  1721,  1884,  2028,  2318,
        2513,  2694,  2702,  2716,  2762,  2841,  2858,  2959,  3243,
        3510,  3949,  5349,  5669,  6377,  7153,  7361,  8622,  8636,
       27369, 44191, 48783, 50068, 58559, 84236])

#### 영화 60에 대한 점수를 pearson 함수를 이용해 구해보겠습니다.

In [8]:
#pearson
print('Predicting values : \t',predict_score(3,60))
print('Real rating value :\t', r_mat[3,60])

Predicting values : 	 (-0.34167287086665615, -0.3467091069788346)
Real rating value :	 -0.5686274509803924


#### 같은 영화에 대한 점수를 cosine similarity로 구해보겠습니다.

In [9]:
#cosine
print('Predicting values : \t',predict_score(3,60,sim_met='cosine'))
print('Real rating value :\t', r_mat[3,60])

Predicting values : 	 (-1.3883074678646374, -1.383321162066603)
Real rating value :	 -0.5686274509803924


#### PCC를 이용한 결과가 조금 더 낫습니다. 영화 247에도 해보겠습니다.

In [10]:
# pearson
print('Predicting values : \t',predict_score(3,247))
print('Real rating value :\t', r_mat[3,247])

Predicting values : 	 (0.15524332555657033, 0.15393480860201272)
Real rating value :	 -0.06862745098039236


In [11]:
#cosine
print('Predicting values : \t',predict_score(3,247,sim_met='cosine'))
print('Real rating value :\t', r_mat[3,247])

Predicting values : 	 (0.20257315610389953, 0.2033382372547101)
Real rating value :	 -0.06862745098039236


#### 영화 247에도 피어슨이 좀 더 나은 결과를 보여주었습니다. 이제 에러를 좀 더 정확하게 측정해보겠습니다.

In [12]:
pearson_ver1_errors = 0
pearson_ver2_errors = 0
cosine_ver1_errors = 0
cosine_ver2_errors = 0

for i in range(100):
    rated_mov = np.array(np.nonzero(r_mat[i,:])).flatten()
    
    for j in rated_mov:
        pred_val = predict_score(i,j,sim_met='pearson')
        
        pearson_ver1_errors = pearson_ver1_errors + np.sqrt(np.square(pred_val[0] - r_mat[i,j]))
        pearson_ver2_errors = pearson_ver2_errors + np.sqrt(np.square(pred_val[1] - r_mat[i,j]))
        
        pred_val = predict_score(i,j,sim_met='cosine')
        
        cosine_ver1_errors = cosine_ver1_errors + np.sqrt(np.square(pred_val[0] - r_mat[i,j]))
        cosine_ver2_errors = cosine_ver2_errors + np.sqrt(np.square(pred_val[1] - r_mat[i,j]))

In [14]:
pearson_ver1_errors, pearson_ver2_errors

(8498.193157341588, 8430.815970617206)